# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO. You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Imports

In [1]:
# Standard libs
import pandas as pd
import numpy as np

In [2]:
# Machine Learning
from sklearn.linear_model import Lasso

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales_dtype={'bathrooms':float, 'waterfront':int, 'sqft_above':int,
             'sqft_living15':float, 'grade':int, 'yr_renovated':int,
             'price':float, 'bedrooms':float, 'zipcode':str, 'long':float,
             'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int,
             'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str,
             'sqft_lot':int, 'view':int}

In [5]:
sales = pd.read_csv("./data/kc_house_data.csv", dtype=sales_dtype)
sales.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [11]:
sales["sqft_living_sqrt"] = np.sqrt(sales["sqft_living"])
sales["sqft_lot_sqrt"] = np.sqrt(sales["sqft_lot"])
sales["bedrooms_square"] = np.square(sales["bedrooms"])

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales["floors"] = sales["floors"].astype(float) 
sales["floors_square"] = np.square(sales["floors"])

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [99]:
all_features = ["bedrooms", "bedrooms_square", "bathrooms",
                "sqft_living", "sqft_living_sqrt", "sqft_lot",
                "sqft_lot_sqrt", "floors", "floors_square",
                "waterfront", "view", "condition", "grade", "sqft_above",
                "sqft_basement", "yr_built", "yr_renovated"]

Using the entire house dataset, learn Lasso regression weights using an L1 penalty of 5e2. Make sure to add "normalize=True" when creating the Lasso model.

In [13]:
model_all = Lasso(alpha=5e2, normalize=True)
model_all.fit(sales[all_features], sales["price"])

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

### Find what features had non-zero weight.

In [87]:
print("Number of nonzeros features:",np.count_nonzero(np.append(model_all.intercept_,model_all.coef_)))

Number of nonzeros features: 4


** Showing weight for all features **

In [108]:
weight_index = all_features.copy()
weight_index.insert(0,"intercept")
features_weight = pd.DataFrame(np.append(model_all.intercept_,model_all.coef_), columns=["weight"], index=weight_index)
print(features_weight)

                         weight
intercept        -218136.214035
bedrooms               0.000000
bedrooms_square        0.000000
bathrooms              0.000000
sqft_living          134.439314
sqft_living_sqrt       0.000000
sqft_lot               0.000000
sqft_lot_sqrt          0.000000
floors                 0.000000
floors_square          0.000000
waterfront             0.000000
view               24750.004586
condition              0.000000
grade              61749.103091
sqft_above             0.000000
sqft_basement          0.000000
yr_built              -0.000000
yr_renovated           0.000000


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [106]:
nonzero_features = features_weight[features_weight["weight"] != 0]
print(nonzero_features)

                    weight
intercept   -218136.214035
sqft_living     134.439314
view          24750.004586
grade         61749.103091


# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

### Loading the dataset

Loading the dataset from files provided for the class as not using GraphLab.

Let's redefine our **sales_dtype** so that [floors] is read as a float.

In [31]:
sales_dtype={'bathrooms':float, 'waterfront':int, 'sqft_above':int,
             'sqft_living15':float, 'grade':int, 'yr_renovated':int,
             'price':float, 'bedrooms':float, 'zipcode':str, 'long':float,
             'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int,
             'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str,
             'sqft_lot':int, 'view':int}

In [32]:
sales_train = pd.read_csv("./data/wk3_kc_house_train_data.csv", dtype=sales_dtype)
sales_valid = pd.read_csv("./data/wk3_kc_house_valid_data.csv", dtype=sales_dtype)
sales_test = pd.read_csv("./data/wk3_kc_house_test_data.csv", dtype=sales_dtype)

### Adding 4 new features as previously

In [33]:
# Training set
sales_train["sqft_living_sqrt"] = np.sqrt(sales_train["sqft_living"])
sales_train["sqft_lot_sqrt"] = np.sqrt(sales_train["sqft_lot"])
sales_train["bedrooms_square"] = np.square(sales_train["bedrooms"])
sales_train["floors_square"] = np.square(sales_train["floors"])

# Validation set
sales_valid["sqft_living_sqrt"] = np.sqrt(sales_valid["sqft_living"])
sales_valid["sqft_lot_sqrt"] = np.sqrt(sales_valid["sqft_lot"])
sales_valid["bedrooms_square"] = np.square(sales_valid["bedrooms"])
sales_valid["floors_square"] = np.square(sales_valid["floors"])

# Test set
sales_test["sqft_living_sqrt"] = np.sqrt(sales_test["sqft_living"])
sales_test["sqft_lot_sqrt"] = np.sqrt(sales_test["sqft_lot"])
sales_test["bedrooms_square"] = np.square(sales_test["bedrooms"])
sales_test["floors_square"] = np.square(sales_test["floors"])

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Make sure to specify
normalize=True in the constructor
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

Create an empty frame to hold the RSS for each value of l1_penalty

In [65]:
log_range = np.logspace(1, 7, num=13)
rss_valid_df = pd.DataFrame(np.zeros((log_range.shape[0],1)), index=log_range, columns=["RSS_Valid"])

In [68]:
for l1_penalty in log_range:
    model = Lasso(alpha=l1_penalty, normalize=True)
    model.fit(sales_train[all_features], sales_train["price"])
    rss_valid_df.loc[l1_penalty] = (np.square(model.predict(sales_valid[all_features]) - sales_valid["price"]).sum())

In [69]:
rss_valid_df

,RSS_Valid
1.000000e+01,3.982133e+14
3.162278e+01,3.990419e+14
1.000000e+02,4.297916e+14
3.162278e+02,4.637398e+14
1.000000e+03,6.458987e+14
3.162278e+03,1.222507e+15
1.000000e+04,1.222507e+15
3.162278e+04,1.222507e+15
1.000000e+05,1.222507e+15
3.162278e+05,1.222507e+15


*** QUIZ QUESTIONS ***
* **What was the best value for the `l1_penalty`?**


In [76]:
print("RSS min value is:", rss_valid_df["RSS_Valid"].min(),"for l1_penalty:",rss_valid_df["RSS_Valid"].idxmin())

RSS min value is: 3.982133273e+14 for l1_penalty: 10.0


* **What is the RSS on TEST data of the model with the best `l1_penalty`?**

Re-fit the model with l1_penalty = **`rss_valid_df["RSS_Valid"].idxmin()`**

In [84]:
model = Lasso(alpha=rss_valid_df["RSS_Valid"].idxmin(), normalize=True)
model.fit(sales_train[all_features], sales_train["price"])
rss_test = np.square(model.predict(sales_test[all_features]) - sales_test["price"]).sum()
print("rss_test:", rss_test)

rss_test: 98467402552698.81


***QUIZ QUESTION***
* **Also, using this value of L1 penalty, how many nonzero weights do you have (this is including the intercept!)?**

In [110]:
print("Number of nonzeros features:",np.count_nonzero(np.append(model_all.intercept_,model_all.coef_)))

Number of nonzeros features: 4


In [109]:
weight_index = all_features.copy()
weight_index.insert(0,"intercept")
features_weight = pd.DataFrame(np.append(model.intercept_,model.coef_), columns=["weight"], index=weight_index)
print(features_weight)

                        weight
intercept         6.630156e+06
bedrooms         -1.614456e+04
bedrooms_square   3.732454e+02
bathrooms         5.084124e+04
sqft_living       6.178536e+02
sqft_living_sqrt -4.441135e+04
sqft_lot          7.856231e-01
sqft_lot_sqrt    -7.011948e+02
floors           -0.000000e+00
floors_square     5.014200e+03
waterfront        6.194888e+05
view              3.804186e+04
condition         2.499877e+04
grade             1.287162e+05
sqft_above        0.000000e+00
sqft_basement     0.000000e+00
yr_built         -3.293831e+03
yr_renovated      1.005732e+01


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [ ]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [ ]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [ ]:
l1_penalty_min = 
l1_penalty_max = 

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?